In [3]:
import subprocess
def get_h264(video_pth,output_pth):
    command = ['ffmpeg', '-i', video_pth, '-c:v', 'libx264', output_pth]

    try:
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
    except subprocess.CalledProcessError as e:
        print("FFmpeg returned an error:", e.stderr)

In [21]:
import json
import pandas as pd
def get_meta_data(video_pth,output_pth,json_pth):
    get_h264(video_pth,output_pth)
    command = ["ffprobe", "-select_streams", "v:0", "-show_frames", "-print_format", "json", output_pth]
    
    with open(json_pth, "w") as file:
        subprocess.call(command, stdout=file)
    with open(json_pth, 'r') as fh:
        jd1 = json.load(fh)
    return pd.DataFrame(jd1["frames"])

In [22]:
import glob
train_path="../Test/anomaly/*.mp4"
files=glob.glob(train_path)
files

['../Test/anomaly\\1.mp4',
 '../Test/anomaly\\2.mp4',
 '../Test/anomaly\\3.mp4',
 '../Test/anomaly\\4.mp4',
 '../Test/anomaly\\5.mp4',
 '../Test/anomaly\\6.mp4']

In [35]:
from threading import Thread
csv_pth="../Test/csv/anomaly/"
dfs={}
def my_iter(pth):
    global dfs
    json_pth=f"{pth}.json"
    output_pth=f"{pth}.h264"
    df=get_meta_data(pth,output_pth,json_pth)
    name=pth.split('\\')[-1]
    df.to_csv(f"{csv_pth}{name}.csv")
    dfs[name]=df

tasks=[Thread(target=my_iter,args=(pth,)) for pth in files]
for t in tasks:
    t.start()
for t in tasks:
    print("joining")
    t.join()
    

joining
joining
joining
joining
joining
joining


In [42]:
dfs['1.mp4']

,media_type,stream_index,key_frame,duration,duration_time,pkt_pos,pkt_size,width,height,crop_top,crop_bottom,crop_left,crop_right,pix_fmt,pict_type,interlaced_frame,top_field_first,repeat_pict,chroma_location,side_data_list
0,video,0,1,48000,0.040000,0,64295,800,600,0,0,0,0,yuv420p,I,0,0,0,left,[{'side_data_type': 'H.26[45] User Data Unregi...
1,video,0,0,48000,0.040000,69638,583,800,600,0,0,0,0,yuv420p,B,0,0,0,left,NaN
2,video,0,0,48000,0.040000,64295,5343,800,600,0,0,0,0,yuv420p,P,0,0,0,left,NaN
3,video,0,0,48000,0.040000,77592,595,800,600,0,0,0,0,yuv420p,B,0,0,0,left,NaN
4,video,0,0,48000,0.040000,76683,909,800,600,0,0,0,0,yuv420p,B,0,0,0,left,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19256,video,0,0,48000,0.040000,50201245,579,800,600,0,0,0,0,yuv420p,B,0,0,0,left,NaN
19257,video,0,0,48000,0.040000,50202160,258,800,600,0,0,0,0,yuv420p,B,0,0,0,left,NaN
19258,video,0,0,48000,0.040000,50199764,1481,800,600,0,0,0,0,yuv420p,P,0,0,0,left,NaN
19259,video,0,0,48000,0.040000,50202418,576,800,600,0,0,0,0,yuv420p,P,0,0,0,left,NaN


In [43]:
dfs['1.mp4'].drop("side_data_list",axis=1).nunique()

media_type              1
stream_index            1
key_frame               2
duration                1
duration_time           1
pkt_pos             19261
pkt_size             6130
width                   1
height                  1
crop_top                1
crop_bottom             1
crop_left               1
crop_right              1
pix_fmt                 1
pict_type               3
interlaced_frame        1
top_field_first         1
repeat_pict             1
chroma_location         1
dtype: int64

In [44]:
dfs['1.mp4'].columns

Index(['media_type', 'stream_index', 'key_frame', 'duration', 'duration_time',
       'pkt_pos', 'pkt_size', 'width', 'height', 'crop_top', 'crop_bottom',
       'crop_left', 'crop_right', 'pix_fmt', 'pict_type', 'interlaced_frame',
       'top_field_first', 'repeat_pict', 'chroma_location', 'side_data_list'],
      dtype='object')

In [105]:
dfs_new={}
for name, df in dfs.items():
    df_new=df[['pkt_pos', 'pkt_size','pict_type']].copy()
    df_new.loc[:,'pkt_pos']=df_new['pkt_pos'].astype(int)
    df_new.loc[:,'pkt_size']=df_new['pkt_size'].astype(int)
    df_new["time_pos"]=df_new.index/25
    df_new["type"]=df_new["pict_type"]
    df_new["byte_pos"]=df_new["pkt_pos"]
    df_new["size"]=df_new["pkt_size"]/df_new["pkt_size"].max()
    df_new.drop(["pkt_pos","pict_type"],axis=1,inplace=True)
    dfs_new[name]=df_new
    
    

In [106]:
dfs_new.keys()

dict_keys(['1.mp4', '4.mp4', '5.mp4', '6.mp4', '3.mp4', '2.mp4'])

In [107]:
timepoints={
'1.mp4':{"has_move":	"1.38-1.43, 4.33-4.36, 8.41-8.43, 9.11-9.13", 
	"has_blur":	"0.14-0.32, 02.16-03.44, 07.18-07.33, 07.59-08.23, 10.40-11.58",
	"has_block":	"01.00-01.09, 01.21-01.26, 03.45-03.50, 04.08-04.12, 05.07-05.18, 09.42-09.54, 10.21-10.30, 12.20-12.31",
	"has_overexposure":	"01.09-01.11, 03.50-03.53, 04.12-04.15, 09.54-09.58, 10.30-10.34, 12.31-12.34"},

'2.mp4':	{"has_block":	"0.42-0.53, 02.15-02.26, 03.22-03.33, 04.15-04.22",
	"has_overexposure":	"0.42-0.53, 03.22-03.33"},

'3.mp4':{	"has_block":	"02.05-02.11, 04.09-04.18",
	"has_overexposure":	"01.03-01.11, 03.05-03.16"},

'4.mp4':{	 "has_block":	"01.09-01.16, 03.27-03.33",
	"has_overexposure":	"02.13-02.20, 04.17-04.25"},

'5.mp4':{	"has_block":	"01.11-01.19, 02.01-02.14, 02.23-02.30, 03.04-03.10, 04.05-04.15",
	"has_overexposure":	"01.19-01.21, 02.01-02.14, 02.23-02.30, 03.10-03.12, 04.05-04.16"},

'6.mp4':{	"has_block":	"01.03-01.09, 03.02-03.12, 04.01-04.20",
	"has_overexposure":	"02.01-02.09, 04.01-04.20"}}

In [108]:
def get_sec(s):
    a,b=map(int,s.split("."))
    return a*60+b
def get_times(s):
    times=[x for x in s.replace(","," ").split(" ") if x!='']
    aa=[]
    for t in times:
        a=tuple(map(get_sec,t.split("-")))
        aa.append(a)
    return aa
        
        
for file, anomalies in timepoints.items():
    for  anomaly, timecodes in anomalies.items():
        timepoints[file][anomaly]=get_times(timecodes)
        

In [109]:
timepoints

{'1.mp4': {'has_move': [(98, 103), (273, 276), (521, 523), (551, 553)],
  'has_blur': [(14, 32), (136, 224), (438, 453), (479, 503), (640, 718)],
  'has_block': [(60, 69),
   (81, 86),
   (225, 230),
   (248, 252),
   (307, 318),
   (582, 594),
   (621, 630),
   (740, 751)],
  'has_overexposure': [(69, 71),
   (230, 233),
   (252, 255),
   (594, 598),
   (630, 634),
   (751, 754)]},
 '2.mp4': {'has_block': [(42, 53), (135, 146), (202, 213), (255, 262)],
  'has_overexposure': [(42, 53), (202, 213)]},
 '3.mp4': {'has_block': [(125, 131), (249, 258)],
  'has_overexposure': [(63, 71), (185, 196)]},
 '4.mp4': {'has_block': [(69, 76), (207, 213)],
  'has_overexposure': [(133, 140), (257, 265)]},
 '5.mp4': {'has_block': [(71, 79),
   (121, 134),
   (143, 150),
   (184, 190),
   (245, 255)],
  'has_overexposure': [(79, 81),
   (121, 134),
   (143, 150),
   (190, 192),
   (245, 256)]},
 '6.mp4': {'has_block': [(63, 69), (182, 192), (241, 260)],
  'has_overexposure': [(121, 129), (241, 260)]}}

In [110]:
for _,df in dfs_new.items():
    df[["is_normal","has_move",
	"has_blur",
	"has_block",
	"has_overexposure"]]=0,0,0,0,0

In [119]:
for name, df in dfs_new.items():
    print(df.columns)
    df.set_index("time_pos",inplace=True)
    for anomaly, arr in timepoints[name].items():
        for tp in arr:
            df.loc[tp[0]:tp[1],anomaly]=1
    df.reset_index(inplace=True)
    df.loc[df[["has_move",
	"has_blur",
	"has_block",
	"has_overexposure"]].sum(axis=1)==0,"is_normal"]=1
    print(df.shape)
        

Index(['time_pos', 'pkt_size', 'type', 'byte_pos', 'size', 'is_normal',
       'has_move', 'has_blur', 'has_block', 'has_overexposure'],
      dtype='object')
(19261, 10)
Index(['time_pos', 'pkt_size', 'type', 'byte_pos', 'size', 'is_normal',
       'has_move', 'has_blur', 'has_block', 'has_overexposure'],
      dtype='object')
(7462, 10)
Index(['time_pos', 'pkt_size', 'type', 'byte_pos', 'size', 'is_normal',
       'has_move', 'has_blur', 'has_block', 'has_overexposure'],
      dtype='object')
(7336, 10)
Index(['time_pos', 'pkt_size', 'type', 'byte_pos', 'size', 'is_normal',
       'has_move', 'has_blur', 'has_block', 'has_overexposure'],
      dtype='object')
(7115, 10)
Index(['time_pos', 'pkt_size', 'type', 'byte_pos', 'size', 'is_normal',
       'has_move', 'has_blur', 'has_block', 'has_overexposure'],
      dtype='object')
(7516, 10)
Index(['time_pos', 'pkt_size', 'type', 'byte_pos', 'size', 'is_normal',
       'has_move', 'has_blur', 'has_block', 'has_overexposure'],
      dtype=

In [121]:
df_new['6.mp4']

KeyError: '6.mp4'

In [124]:
for name, df in dfs_new.items():
    print(name)
    df.to_csv(f"csv/Test/anomaly/{name}.csv")

1.mp4
4.mp4
5.mp4
6.mp4
3.mp4
2.mp4
